**Step 1. 安裝所需LIB** 僅需要登入後執行一次即可

In [ ]:
# @title 安裝所需LIB
!pip install matplotlib pandas ccxt ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 723.6 kB/s eta 0:00:00
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 14.3 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.7/288.7 kB 12.2 MB/s eta 0:00:00


In [ ]:
# @title Step 2.主要程式邏輯區
from IPython.display import display, clear_output
import ipywidgets as widgets
import matplotlib.pyplot as plt
import pandas as pd
import ccxt
import re
import os
from datetime import datetime
import time
from google.colab import drive

# Function to convert Binance symbols to another exchange's symbols
def convert_to_exchange(symbols, target_exchange='binanceus'):
    def clean_symbol(symbol):
        return re.sub(r'^\d+', '', symbol)

    if target_exchange == 'binanceus':
        return [clean_symbol(symbol.replace('BINANCE:', 'BINANCEUS:').replace('.P', '').replace('USDT', '')) for symbol in symbols]
    elif target_exchange == 'okx':
        return [clean_symbol(symbol.replace('BINANCE:', 'OKX:').replace('.P', '').replace('USDT', '')) for symbol in symbols]
    else:
        return [clean_symbol(symbol) for symbol in symbols]

# Function to calculate the total bars needed based on timeframe and days
def calc_total_bars(time_interval, days, max_window_size=60):
    if 'm' in time_interval:
        minutes_per_bar = int(time_interval.replace('m', ''))
        bars_per_hour = 60 // minutes_per_bar
    elif 'h' in time_interval:
        hours_per_bar = int(time_interval.replace('h', ''))
        bars_per_hour = 1 / hours_per_bar
        minutes_per_bar = hours_per_bar * 60
    else:
        raise ValueError("Unsupported time interval format")

    bars_per_day = bars_per_hour * 24
    total_bars = int(bars_per_day * (days + max_window_size // (24 * bars_per_hour)))

    return total_bars, minutes_per_bar

# Function to search for a symbol in multiple exchanges
def search_symbol_in_exchanges(symbol, exchanges):
    for exchange in exchanges:
        try:
            exchange.fetch_ticker(symbol)
            return exchange
        except ccxt.BaseError:
            continue
    return None

# Function to process input and plot
def process_and_plot(input_string, days, timeframes, k_line_option, mount_drive, show_plots):
    start_time = time.time()  # Start timing
    binance_symbols = re.findall(r'BINANCE:(\w+)', input_string)
    exchange_symbols = convert_to_exchange(binance_symbols, target_exchange='binanceus')

    # Initialize exchanges
    exchange_binance = ccxt.binanceus()
    exchange_okx = ccxt.okx()
    exchange_bybit = ccxt.bybit()
    exchange_bingx = ccxt.bingx()

    exchanges = [exchange_binance, exchange_okx, exchange_bybit, exchange_bingx]

    # Mount Google Drive if the user agrees
    if mount_drive:
        drive.mount('/content/gdrive', force_remount=True)
        base_dir = '/content/gdrive/My Drive/CryptoCharts/'
        if not os.path.exists(base_dir):
            os.makedirs(base_dir)
    else:
        base_dir = '/content/CryptoCharts/'
        if not os.path.exists(base_dir):
            os.makedirs(base_dir)

    current_time = datetime.now().strftime('%Y%m%d_%H%M%S')
    session_dir = os.path.join(base_dir, current_time)
    os.makedirs(session_dir, exist_ok=True)

    progress = widgets.FloatProgress(value=0.0, min=0.0, max=1.0, description='Processing:')
    display(progress)

    total_symbols = len(exchange_symbols)
    processed_symbols = 0

    for symbol in exchange_symbols:
        exchange_symbol = symbol + '/USDT'
        exchange = search_symbol_in_exchanges(exchange_symbol, exchanges)

        if exchange:
            print(f"Symbol {exchange_symbol} found in {exchange.name.upper()}")
            plot_candlestick([symbol], timeframes, exchange, days, k_line_option, session_dir, show_plots)

        processed_symbols += 1
        progress.value = processed_symbols / total_symbols  # Update progress bar

    if len(exchange_symbols) > len(set(exchange_symbols)):
        unavailable_symbols = set(exchange_symbols) - set([symbol])
        plot_unavailable_symbols(unavailable_symbols, session_dir, show_plots)

    end_time = time.time()  # End timing
    total_time = end_time - start_time
    print(f"Total time taken: {total_time:.2f} seconds")  # Display total time taken

def plot_candlestick(symbols, timeframes, exchange, days, k_line_option, session_dir, show_plots):
    max_window_size = 140
    for symbol in symbols:
        for timeframe in timeframes:
            total_bars, minutes_per_bar = calc_total_bars(timeframe, days, max_window_size)
            fig, ax1 = plt.subplots(figsize=(10, 6))

            try:
                ohlcv = exchange.fetch_ohlcv(symbol + '/USDT', timeframe=timeframe, limit=total_bars)
                df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
                df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

                df['ma_30'] = df['close'].rolling(window=30).mean()
                df['ma_45'] = df['close'].rolling(window=45).mean()
                df['ma_60'] = df['close'].rolling(window=60).mean()

                ax2 = ax1.twinx()

                # Plot Normalized Volume on the left axis
                volume_max = df['volume'].max()
                ax1.bar(df['timestamp'], df['volume'] / volume_max, color='blue', alpha=0.3, width=pd.Timedelta(minutes_per_bar, 'm'))
                ax1.set_ylabel('Normalized Volume')

                # Plot Price Data on the right axis
                if k_line_option == 'K_Bars' or k_line_option == 'ALL':
                    for idx, row in df.iterrows():
                        color = 'green' if row['close'] >= row['open'] else 'red'
                        ax2.plot([row['timestamp'], row['timestamp']], [row['low'], row['high']], color='black')
                        ax2.add_patch(plt.Rectangle((row['timestamp'], row['open']), pd.Timedelta(minutes_per_bar, 'm'), row['close'] - row['open'], color=color))

                if not df['ma_30'].isna().all() or not df['ma_45'].isna().all() or not df['ma_60'].isna().all():
                    ax2.plot(df['timestamp'], df['ma_30'], color='#4CAF50', label='MA 30', alpha=0.5)
                    ax2.plot(df['timestamp'], df['ma_45'], color='#FF5252', label='MA 45', alpha=0.5)
                    ax2.plot(df['timestamp'], df['ma_60'], color='#2962FF', label='MA 60', alpha=0.5)

                    handles, labels = ax2.get_legend_handles_labels()
                    if handles:
                        ax2.legend()

                ax2.set_title(f'{symbol} ({timeframe}) - {exchange.name.upper()}', fontsize=10)
                ax2.set_xlabel('Date')
                ax2.set_ylabel('Price')

                fig.suptitle(f"{exchange.name.upper()} - {symbol} - {timeframe.upper()} {'with K-line' if k_line_option != 'No_K_Bars' else 'without K-line'}", fontsize=16)
                plt.tight_layout()

                timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
                k_line_str = "K" if k_line_option != 'No_K_Bars' else "NoK"
                filename = f"{symbol}_{timeframe}_{k_line_str}_{timestamp}.png"
                filepath = os.path.join(session_dir, filename)
                fig.savefig(filepath)

                if show_plots:
                    plt.show()
                else:
                    plt.close(fig)

            except Exception as e:
                print(f"Error fetching data for {symbol}: {e}")
                plt.close(fig)
                continue

def plot_unavailable_symbols(unavailable_symbols, session_dir, show_plots):
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.text(0.5, 0.5, '\n'.join(unavailable_symbols), fontsize=12, ha='center', va='center')
    ax.set_axis_off()

    fig.suptitle(f"Symbols not found in any exchange", fontsize=16)
    plt.tight_layout()

    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f"Unavailable_symbols_{timestamp}.png"
    filepath = os.path.join(session_dir, filename)
    fig.savefig(filepath)

    if show_plots:
        plt.show()
    else:
        plt.close(fig)

# User Interface
text_area = widgets.Textarea(
    value='',
    placeholder='Enter your string here...',
    description='Input:',
    layout=widgets.Layout(width='100%', height='200px')
)

# Dropdown for selecting timeframes
timeframe_dropdown = widgets.SelectMultiple(
    options=['30m', '45m', '1h', '2h', '4h'],
    value=['30m', '1h', '2h'],
    description='Timeframes:',
    disabled=False
)

# Slider for setting days
days_slider = widgets.IntSlider(
    value=1,
    min=1,
    max=30,
    step=1,
    description='Days:',
    continuous_update=False
)

# Dropdown for selecting K-line options
k_line_dropdown = widgets.Dropdown(
    options=['K_Bars', 'No_K_Bars', 'ALL'],
    value='No_K_Bars',
    description='Chart Options:',
    disabled=False
)

# Checkbox for mounting Google Drive
mount_drive_checkbox = widgets.Checkbox(
    value=True,
    description='Mount Google Drive',
    disabled=False
)

# Checkbox for showing plots in the interface
show_plots_checkbox = widgets.Checkbox(
    value=True,
    description='Show Plots in Interface',
    disabled=False
)

# Button to trigger the processing
button = widgets.Button(description="Generate Charts")

def on_button_click(b):
    clear_output(wait=True)
    display(text_area, timeframe_dropdown, days_slider, k_line_dropdown, mount_drive_checkbox, show_plots_checkbox, button)
    process_and_plot(
        input_string=text_area.value,
        days=days_slider.value,
        timeframes=list(timeframe_dropdown.value),
        k_line_option=k_line_dropdown.value,
        mount_drive=mount_drive_checkbox.value,
        show_plots=show_plots_checkbox.value
    )

button.on_click(on_button_click)

# Display the widgets
display(text_area, timeframe_dropdown, days_slider, k_line_dropdown, mount_drive_checkbox, show_plots_checkbox, button)


Textarea(value='###大盤,BINANCE:BTCUSDT.P,BINANCE:ETHUSDT.P,BINANCE:BNBUSDT.P,\n###強勢族群,BINANCE:BAKEUSDT.P,BINAN…

SelectMultiple(description='Timeframes:', index=(0, 1, 2, 3, 4), options=('30m', '45m', '1h', '2h', '4h'), val…

IntSlider(value=1, continuous_update=False, description='Days:', max=30, min=1)

Dropdown(description='Chart Options:', index=1, options=('K_Bars', 'No_K_Bars', 'ALL'), value='No_K_Bars')

Checkbox(value=True, description='Mount Google Drive')

Checkbox(value=False, description='Show Plots in Interface')

Button(description='Generate Charts', style=ButtonStyle())

Mounted at /content/gdrive


FloatProgress(value=0.0, description='Processing:', max=1.0)

Symbol BTC/USDT found in BINANCE US
Error fetching data for BTC: binanceus {"code":-1120,"msg":"Invalid interval."}
Symbol ETH/USDT found in BINANCE US
Error fetching data for ETH: binanceus {"code":-1120,"msg":"Invalid interval."}
Symbol BNB/USDT found in BINANCE US
Error fetching data for BNB: binanceus {"code":-1120,"msg":"Invalid interval."}
Symbol BAKE/USDT found in BINGX
Error fetching data for BAKE: bingx {"code":100400,"msg":"unknown interval 45m","timestamp":1723081976229}
Symbol XLM/USDT found in BINANCE US
Error fetching data for XLM: binanceus {"code":-1120,"msg":"Invalid interval."}
Symbol XRP/USDT found in BINANCE US
Error fetching data for XRP: binanceus {"code":-1120,"msg":"Invalid interval."}
Symbol SFP/USDT found in BINGX
Error fetching data for SFP: bingx {"code":100400,"msg":"unknown interval 45m","timestamp":1723081985591}
Symbol ZEC/USDT found in BINANCE US
Error fetching data for ZEC: binanceus {"code":-1120,"msg":"Invalid interval."}
Symbol STORJ/USDT found in B